In [1]:
import pandas as pd

In [2]:
GAME_ID = '0022400630'

In [3]:
from nba_api.live.nba.endpoints import PlayByPlay

PlayByPlay?

Init signature: PlayByPlay(game_id, proxy=None, headers=None, timeout=30, get_request=True)
Docstring:      <no docstring>
File:           ~/anaconda3/envs/MBAI/lib/python3.13/site-packages/nba_api/live/nba/endpoints/playbyplay.py
Type:           type
Subclasses:     

In [4]:
pbp = PlayByPlay(GAME_ID)

In [118]:
df = pd.DataFrame(pbp.get_dict()['game']['actions'])

In [119]:
df.columns

Index(['actionNumber', 'clock', 'timeActual', 'period', 'periodType',
       'actionType', 'subType', 'qualifiers', 'personId', 'x', 'y',
       'possession', 'scoreHome', 'scoreAway', 'edited', 'orderNumber',
       'isTargetScoreLastPeriod', 'xLegacy', 'yLegacy', 'isFieldGoal', 'side',
       'description', 'personIdsFilter', 'teamId', 'teamTricode', 'descriptor',
       'jumpBallRecoveredName', 'jumpBallRecoverdPersonId', 'playerName',
       'playerNameI', 'jumpBallWonPlayerName', 'jumpBallWonPersonId',
       'jumpBallLostPlayerName', 'jumpBallLostPersonId', 'area', 'areaDetail',
       'shotDistance', 'shotResult', 'blockPlayerName', 'blockPersonId',
       'shotActionNumber', 'reboundTotal', 'reboundDefensiveTotal',
       'reboundOffensiveTotal', 'pointsTotal', 'assistPlayerNameInitial',
       'assistPersonId', 'assistTotal', 'officialId', 'foulPersonalTotal',
       'foulTechnicalTotal', 'foulDrawnPlayerName', 'foulDrawnPersonId',
       'turnoverTotal', 'stealPlayerName', 's

In [120]:
cols2drop = [
    'actionNumber', 'periodType', 'edited', 'orderNumber',
    'isTargetScoreLastPeriod', 'isFieldGoal', 'side', 'personIdsFilter',
    'teamTricode', 'jumpBallRecoveredName', 'playerName',
    'playerNameI', 'jumpBallWonPlayerName', 'jumpBallLostPlayerName', 
    'blockPlayerName', 'shotActionNumber', 'reboundTotal', 'reboundDefensiveTotal',
    'reboundOffensiveTotal', 'pointsTotal', 'assistPlayerNameInitial',
    'assistTotal', 'foulPersonalTotal', 'foulTechnicalTotal', 
    'foulDrawnPlayerName', 'turnoverTotal', 'stealPlayerName'
]

df.drop(columns=cols2drop, inplace=True)

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   clock                     611 non-null    object 
 1   timeActual                611 non-null    object 
 2   period                    611 non-null    int64  
 3   actionType                611 non-null    object 
 4   subType                   611 non-null    object 
 5   qualifiers                611 non-null    object 
 6   personId                  611 non-null    int64  
 7   x                         164 non-null    float64
 8   y                         164 non-null    float64
 9   possession                611 non-null    int64  
 10  scoreHome                 611 non-null    object 
 11  scoreAway                 611 non-null    object 
 12  xLegacy                   164 non-null    float64
 13  yLegacy                   164 non-null    float64
 14  descriptio

In [152]:
df[['clock', 'timeActual']]

,clock,timeActual
0,0 days 00:12:00,2025-01-25 00:10:48.500000+00:00
1,0 days 00:11:57,2025-01-25 00:10:50.500000+00:00
2,0 days 00:11:37,2025-01-25 00:11:10.200000+00:00
3,0 days 00:11:37,2025-01-25 00:11:10.200000+00:00
4,0 days 00:11:34,2025-01-25 00:11:13.200000+00:00
...,...,...
606,0 days 00:00:10.700000,2025-01-25 02:30:23.300000+00:00
607,0 days 00:00:03.300000,2025-01-25 02:30:35.800000+00:00
608,0 days 00:00:00,2025-01-25 02:30:44.700000+00:00
609,0 days 00:00:00,2025-01-25 02:30:53.700000+00:00


This is not giving back the correct result as 'ms'.

In [159]:
df['clock'] = pd.to_timedelta(df['clock'], unit='ms')
df['timeActual'] = pd.to_datetime(df['timeActual'], unit='ms')

In [124]:
df['period'].unique()

array([1, 2, 3, 4])

In [125]:
df['period'] = df['period'].astype('uint8')

In [126]:
df['actionType'].unique()

array(['period', 'jumpball', '2pt', 'block', 'rebound', '3pt', 'foul',
       'substitution', 'timeout', 'turnover', 'steal', 'freethrow',
       'violation', 'game'], dtype=object)

In [127]:
action_types = [action for action in df['actionType'].unique() if action != '']
action_subtypes = {
    action: action_df['subType'].unique().tolist()
        for action, action_df in df.groupby('actionType') if action != ''
}

action_subtypes

{'2pt': ['DUNK', 'Jump Shot', 'Layup', 'Hook'],
 '3pt': ['Jump Shot'],
 'block': [''],
 'foul': ['personal', 'offensive', 'technical'],
 'freethrow': ['1 of 2', '2 of 2', '1 of 1'],
 'game': ['end'],
 'jumpball': ['recovered'],
 'period': ['start', 'end'],
 'rebound': ['defensive', 'offensive'],
 'steal': [''],
 'substitution': ['out', 'in'],
 'timeout': ['full'],
 'turnover': ['out-of-bounds',
  'traveling',
  'lost ball',
  'bad pass',
  'offensive foul',
  'shot clock'],
 'violation': ['kicked ball',
  'defensive goaltending',
  'lane',
  'delay-of-game']}

In [128]:
df[['actionType', 'subType', 'qualifiers', 'description', 'descriptor']]

,actionType,subType,qualifiers,description,descriptor
0,period,start,[],Period Start,NaN
1,jumpball,recovered,[],Jump Ball M. Williams vs. D. Clingan: Tip to V...,startperiod
2,2pt,DUNK,[pointsinthepaint],MISS M. Williams cutting DUNK - blocked,cutting
3,block,,[],D. Clingan BLOCK (1 BLK),NaN
4,rebound,defensive,[],D. Clingan REBOUND (Off:0 Def:1),NaN
...,...,...,...,...,...
606,2pt,DUNK,"[pointsinthepaint, 2ndchance]",M. Williams cutting DUNK (13 PTS) (J. Green 2 ...,cutting
607,2pt,DUNK,[pointsinthepaint],T. Camara driving DUNK (11 PTS) (A. Simons 2 AST),driving
608,2pt,Layup,[pointsinthepaint],M. Williams cutting Layup (15 PTS) (N. Smith J...,cutting
609,period,end,[],Period End,NaN


In [129]:
qualifiers_set = set()
for qualifiers in df['qualifiers']:
    for qualifier in qualifiers: 
        qualifiers_set.add(qualifier)

qualifiers_set

{'1freethrow',
 '2freethrow',
 '2ndchance',
 'deadball',
 'defensivegoaltending',
 'fastbreak',
 'fromturnover',
 'inpenalty',
 'mandatory',
 'pointsinthepaint',
 'startperiod',
 'team'}

In [130]:
df['descriptor'].unique()

array([nan, 'startperiod', 'cutting', 'pullup', 'running', 'turnaround',
       'driving floating bank', 'putback', 'step back',
       'running alley-oop', 'driving floating', 'running pullup',
       'bad pass', 'tip', 'driving finger roll', 'driving', 'shooting',
       'turnaround fadeaway', 'charge', 'alley-oop', 'loose ball',
       'lost ball', 'floating', 'driving reverse', 'defensive-3-second',
       'technical', 'running finger roll', 'take'], dtype=object)

In [131]:
df[['x', 'y', 'xLegacy', 'yLegacy']]

,x,y,xLegacy,yLegacy
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,8.886334,51.960784,-10.0,31.0
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
606,94.037451,51.715686,9.0,4.0
607,7.440867,52.696078,-13.0,17.0
608,94.431669,47.058824,-15.0,0.0
609,NaN,NaN,NaN,NaN


In [132]:
assert all(df['x'].dropna() >= 0) and all(df['y'].dropna() >= 0)
assert all(df['xLegacy'].dropna() == df['xLegacy'].dropna().astype('int')) and all(df['yLegacy'].dropna() == df['yLegacy'].dropna().astype('int'))

In [133]:
df.drop(columns=['xLegacy', 'yLegacy'], inplace=True)

df['x'] = df['x'].fillna(-1).astype('float16')
df['y'] = df['y'].fillna(-1).astype('float16')

In [134]:
df['area'].unique()

array([nan, 'Restricted Area', 'Mid-Range', 'In The Paint (Non-RA)',
       'Above the Break 3', 'Left Corner 3', 'Right Corner 3'],
      dtype=object)

In [135]:
df['areaDetail'].unique()

array([nan, '0-8 Center', '8-16 Right', '8-16 Center', '24+ Right Center',
       '24+ Left', '24+ Right', '16-24 Right Center', '8-16 Left',
       '24+ Left Center', '16-24 Center', '24+ Center', '16-24 Right',
       '16-24 Left Center'], dtype=object)

In [136]:
df['shotResult'].unique()

array([nan, 'Missed', 'Made'], dtype=object)

In [137]:
df['shotResult'] = df['shotResult'].replace({'Made': True, 'Missed': False, 'nan': pd.NA})

In [138]:
df.isna().sum()

clock                         0
timeActual                    0
period                        0
actionType                    0
subType                       0
qualifiers                    0
personId                      0
x                             0
y                             0
possession                    0
scoreHome                     0
scoreAway                     0
description                   0
teamId                        9
descriptor                  454
jumpBallRecoverdPersonId    610
jumpBallWonPersonId         610
jumpBallLostPersonId        610
area                        233
areaDetail                  233
shotDistance                447
shotResult                  380
blockPersonId               599
assistPersonId              574
officialId                  547
foulDrawnPersonId           562
stealPersonId               590
dtype: int64

In [139]:
df['officialId'].unique()

array([     nan, 1626301., 1628487.,  204059.])

In [140]:
df['officialId'] = df['officialId'].fillna(0).astype('uint32')

In [141]:
df['personId'] = df['personId'].astype('uint32')
df['teamId'] = df['teamId'].fillna(0).astype('uint32')
df['possession'] = df['possession'].astype('uint32')
df['jumpBallRecoverdPersonId'] = df['jumpBallRecoverdPersonId'].fillna(0).astype('uint32')
df['jumpBallWonPersonId'] = df['jumpBallWonPersonId'].fillna(0).astype('uint32')
df['jumpBallLostPersonId'] = df['jumpBallLostPersonId'].fillna(0).astype('uint32')
df['assistPersonId'] = df['assistPersonId'].fillna(0).astype('uint32')
df['blockPersonId'] = df['blockPersonId'].fillna(0).astype('uint32')
df['stealPersonId'] = df['stealPersonId'].fillna(0).astype('uint32')
df['foulDrawnPersonId'] = df['foulDrawnPersonId'].fillna(0).astype('uint32')

df['scoreHome'] = df['scoreHome'].astype('uint8')
df['scoreAway'] = df['scoreAway'].astype('uint8')

df['shotDistance'] = df['shotDistance'].fillna(-1).astype('float16')

In [157]:
df.dtypes

clock                           timedelta64[ns]
timeActual                  datetime64[ns, UTC]
period                                    uint8
actionType                               object
subType                                  object
qualifiers                               object
personId                                 uint32
x                                       float16
y                                       float16
possession                               uint32
scoreHome                                 uint8
scoreAway                                 uint8
description                              object
teamId                                   uint32
descriptor                               object
jumpBallRecoverdPersonId                 uint32
jumpBallWonPersonId                      uint32
jumpBallLostPersonId                     uint32
area                                     object
areaDetail                               object
shotDistance                            

In [160]:
original_memory = pd.DataFrame(pbp.get_dict()['game']['actions']).memory_usage(deep=True).sum()
optimized_memory = df.memory_usage(deep=True).sum()

print(f"Original teams df memory: {original_memory / 1024**2:.2f} MB")
print(f"Optimized teams df memory: {optimized_memory / 1024**2:.2f} MB")
print(f"Memory saved: {(original_memory - optimized_memory) / 1024**2:.2f} MB ({(1 - optimized_memory/original_memory)*100:.1f}%)")

Original teams df memory: 0.91 MB
Optimized teams df memory: 0.30 MB
Memory saved: 0.61 MB (67.2%)
